In [1]:
import cv2
import glob
import random
import math
import numpy as np
import dlib
import itertools
from sklearn.svm import SVC
import pickle

filename = '/Users/cmeaton/Documents/code/ds/METIS/sea19_ds7_workingdir/project_5/src/models/lin_svm_model.sav'

loaded_model = pickle.load(open(filename, 'rb'))

In [2]:
loaded_model

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [54]:
path = '/Users/cmeaton/Documents/code/ds/METIS/sea19_ds7_workingdir/project_5/data/external/outside_testing_images/download.png'
path1 = '/Users/cmeaton/Documents/code/ds/METIS/sea19_ds7_workingdir/project_5/data/external/cohn_dataset/sorted_set/disgust/02_009_00000015.png'
faceDet = cv2.CascadeClassifier('/Users/cmeaton/Documents/code/opencv/data/haarcascades/haarcascade_frontalface_default.xml')
faceDet_two = cv2.CascadeClassifier("/Users/cmeaton/Documents/code/opencv/data/haarcascades/haarcascade_frontalface_alt2.xml")
faceDet_three = cv2.CascadeClassifier("/Users/cmeaton/Documents/code/opencv/data/haarcascades/haarcascade_frontalface_alt.xml")
faceDet_four = cv2.CascadeClassifier("/Users/cmeaton/Documents/code/opencv/data/haarcascades/haarcascade_frontalface_alt_tree.xml")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/Users/cmeaton/Documents/code/ds/METIS/sea19_ds7_workingdir/project_5/src/models/shape_predictor_68_face_landmarks.dat")
data = {} #Make dictionary for all values
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

def process_image(image_path):

    frame = cv2.imread(image_path)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = clahe.apply(gray)
    
    #Detect face using 4 different classifiers
    face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face_two = faceDet_two.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face_three = faceDet_three.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face_four = faceDet_four.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)

    #Go over detected faces, stop at first detected face, return empty if no face.
    if len(face) == 1:
        facefeatures = face
    elif len(face_two) == 1:
        facefeatures = face_two
    elif len(face_three) == 1:
        facefeatures = face_three
    elif len(face_four) == 1:
        facefeatures = face_four
    else:
        facefeatures = ""

    #Cut and save face
    for (x, y, w, h) in facefeatures: #get coordinates and size of rectangle containing face
        gray = gray[y:y+h, x:x+w] #Cut the frame to size
        #print(gray.shape)
        try:
            out = cv2.resize(gray, (350, 350)) #Resize face so all images have same size
            #print(out.shape)
        except:
            pass #If error, pass file
        return out


def get_landmarks(image):
    '''This function locates facial landmarks and computes the relative distance from the mean for each point.'''
   

    training_data = []
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(w)
            landmarks_vectorised.append(z)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append((math.atan2(y, x)*360)/(2*math.pi))
        data['landmarks_vectorised'] = landmarks_vectorised
    if len(detections) < 1:
        data['landmarks_vestorised'] = "error"
       
    training_data.append(data['landmarks_vectorised'])
    return training_data


In [76]:
loaded_model.predict_proba(get_landmarks(process_image(path1)))


array([[0.09650627, 0.0547831 , 0.3746276 , 0.01241157, 0.40252017,
        0.05915127]])

In [80]:
pic(path)

() () () ()
